In [20]:
import pandas as pd

# --- 1. 파일 로드 (변수명을 목적에 맞게 변경) ---
peach_yield_path = '시도별_연도별_복숭아_면적_생산량.csv'
temp_max_path = '시도별_생육기_최고기온.csv'
cold_stress_path = '냉해_스트레스.csv'
heatwave_days_path = '시도별_폭염일수_평균.csv'
rain_avg_path = '시도별_장마_평균_일강수량.csv'

# 데이터 로드
try:
    df_yield = pd.read_csv(peach_yield_path)
    df_temp_max = pd.read_csv(temp_max_path)
    df_cold_stress = pd.read_csv(cold_stress_path)
    df_heatwave_days = pd.read_csv(heatwave_days_path)
    df_rain_avg = pd.read_csv(rain_avg_path)
except FileNotFoundError as e:
    print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인해주세요. -> {e}")
    # return


# --- 2. 컬럼명 통일 및 데이터 정리 ---

# df_yield: '시도' → '시도명'
if '시도' in df_yield.columns:
    df_yield.rename(columns={'시도': '시도명'}, inplace=True)
elif '시도별' in df_yield.columns: # 이전 추정치도 함께 확인
    df_yield.rename(columns={'시도별': '시도명'}, inplace=True)

# df_temp_max, df_cold_stress, df_heatwave_days: year -> 연도
df_temp_max.rename(columns={'year': '연도'}, inplace=True)
df_cold_stress.rename(columns={'year': '연도'}, inplace=True)
df_heatwave_days.rename(columns={'year': '연도'}, inplace=True)

# df_rain_avg: 시도별 → 시도명, year → 연도
df_rain_avg.rename(columns={'시도별': '시도명'}, inplace=True)
df_rain_avg.rename(columns={'year': '연도'}, inplace=True)

# 모든 데이터프레임에서 dtype 통일
for df in [df_yield, df_temp_max, df_cold_stress, df_heatwave_days, df_rain_avg]:
    if '연도' in df.columns:
        df['연도'] = df['연도'].astype(str)
    if '시도명' in df.columns:
        df['시도명'] = df['시도명'].astype(str)

# --- 3. 데이터 병합 (Merge) ---

df_merged_step1 = pd.merge(df_yield, df_temp_max, on=['연도', '시도명'], how='inner')
df_merged_step2 = pd.merge(df_merged_step1, df_cold_stress, on=['연도', '시도명'], how='inner')
df_merged_step3 = pd.merge(df_merged_step2, df_heatwave_days, on=['연도', '시도명'], how='inner')
df_final = pd.merge(df_merged_step3, df_rain_avg, on=['연도', '시도명'], how='inner')

# --- 4. 중간 결과 저장 (선택 사항) ---
# df_final.to_csv('./data/복숭아_회귀_데이터셋_2단계.csv', index=False, encoding='utf-8')


# --- 5. 일조시간 데이터 추가 병합 (수정된 부분) ---

# ✅ 파일명을 '생육기_평균_일조시간.csv'로 수정하여 로드
df_daylight_hours = pd.read_csv("생육기_합계일조시간.csv", encoding='utf-8')

# ❌ 오류 발생 코드 제거 또는 수정 ❌
# df_daylight_hours.drop(columns=['관측소 수', '합계 일조시간(hr)'], axis=1, inplace=True)
# -> 해당 컬럼들이 최종 저장 파일에 없으므로 이 줄은 삭제합니다.

# ✅ 'year' 컬럼이 이미 '연도'로 로드되었을 가능성이 높지만, 확인 차원에선 필요 없는 코드
# df_daylight_hours.rename(columns={'year': '연도'}, inplace=True) 

# 데이터 타입 통일
if '연도' in df_daylight_hours.columns:
    df_daylight_hours['연도'] = df_daylight_hours['연도'].astype(str)
if '시도명' in df_daylight_hours.columns:
    df_daylight_hours['시도명'] = df_daylight_hours['시도명'].astype(str)


# 최종 데이터셋에 일조시간 데이터 병합
df_final_with_daylight = pd.merge(df_final, df_daylight_hours, on=['연도', '시도명'], how='inner')


# --- 6. 최종 결과 저장 ---

final_output_path = '복숭아_회귀_데이터셋_일조시간추가.csv'
df_final_with_daylight.to_csv(final_output_path, index=False, encoding='utf-8')

print(f"✅ 모든 데이터 병합 및 처리가 완료되었습니다.")
print(f"✅ 최종 파일이 '{final_output_path}' 경로에 저장되었습니다.")

✅ 모든 데이터 병합 및 처리가 완료되었습니다.
✅ 최종 파일이 '복숭아_회귀_데이터셋_일조시간추가.csv' 경로에 저장되었습니다.
